# LangChainによるPDF資料の質疑応答システム構築


## 1. ライブラリをインストールします 

In [1]:
!pip install langchain openai chromadb tiktoken pypdf gradio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.8/248.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

openAIからAPI_KEYを取得してこちらに貼って下さい

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "*******"

## 2. ライブラリをインポートします

In [3]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [4]:
llm = OpenAI(temperature=0, max_tokens=-1)
print(llm("tell me a joke"))



Q: What did the fish say when it hit the wall?
A: Dam!


## 3. PDFファイルをインポートします
- 「格付け分類モデルにおける機械学習の応用：機械学習の説明可能性を高める手法」日本銀行ワーキングペーパーシリーズ 2023年3月 橋本龍一郎 三浦翔 吉崎康則



In [5]:
!wget -q https://www.boj.or.jp/research/wps_rev/wps_2023/data/wp23j03.pdf

In [6]:
!ls

sample_data  wp23j03.pdf


## 4. RetrievalQAを使ってQAシステムのパイプラインを構築します 

In [7]:
# 1.PDF資料をロードします (load document)
loader = PyPDFLoader("wp23j03.pdf")
documents = loader.load()
# 2.ドキュメントをチャンクに分割します (split the documents into chunks)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
# 3.埋め込みベクトル用モデルを選択します (select which embeddings we want to use)
embeddings = OpenAIEmbeddings()
# 4.ベクトル格納のためのDBを作成します (create the vectorestore to use as the index)
db = Chroma.from_documents(texts, embeddings)
# 5.リトリバーにより、質問と関連するチャンクを呼び出します　(expose this index in a retriever interface)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# 6.回答を作成するためのchainを作ります（create a chain to answer questions） 
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="map_reduce", retriever=retriever, return_source_documents=True)


In [8]:
query = "このレポートでは、どのような分析手法が用いられてますか? "
result = qa({"query": query+"answer in Japanese"})
result['result']

' このレポートでは、機械学習、順序ロジット回帰、SHAPやPDPなどのXAI（説明可能性を高める枠組み）を用いて分析が行われています。'

In [9]:
query = "このレポートでは、どのような分析手法が用いられてますか? "
result = qa({"query": query+"answer in Japanese"})
result

{'query': 'このレポートでは、どのような分析手法が用いられてますか? answer in Japanese',
 'result': ' 本レポートでは、機械学習を用いた格付け分類モデルの推計と、SHAPやPDPといった機械学習の説明可能性を高める枠組み（XAI）を用いて分析が行われています。',
 'source_documents': [Document(page_content='格付け分類モデルにおける機械学習の 応用： \n機械学習の 説明可能性 を高める 手法 \n橋本龍一郎 * \nryuuichirou.hashimoto@boj.or.jp  \n三浦翔 * \nkakeru.miura @boj.or.jp  \n吉崎康則 * \nyasunori.yoshizaki@boj.or.jp  \nNo.23-J-3 \n2023年3月 日本銀行  \n〒103-8660 日本郵便（株）日本橋郵便局私書箱 30号 \n*金融機構局\n日本銀行ワーキングペーパーシリーズは、日本銀行員および外部研究者の研究成果を\nとりまとめたもので、内外の研究機関、研究者等の有識者から幅広くコメントを頂戴す\nることを意図しています。ただし、論文の中で示された内容や意見は、日本銀行の公式\n見解を示すものではありません。  \nなお、ワーキングペーパーシリーズに対するご意見・ご質問や、掲載ファイルに関する\nお問い合わせは、執筆者までお寄せ下さい。  \n商用目的で転載・複製を行う場合は、予め日本銀行情報サービス局\n(post.prd8@boj.or.jp) までご相談下さい。転載・複製を行う場合は、出所を明記して\n下さい。  \n日本銀行ワーキングペーパーシリーズ', metadata={'source': 'wp23j03.pdf', 'page': 0}),
  Document(page_content='23 デルの予測結果に与える解釈は、 幾つかの重要な 仮定に依拠して いる点には留意が必\n要である。 こうした仮定を把握せずに XAIを機械学習モデルに適用することで、モデ\nルから誤った帰結を得てしまう可能性がある。 XAIを用いる際には、当該手法の仮定\nや前提を把握し検証を行うことが重要と考えられる。  \nまた、本章では説明を簡便にす

In [10]:
query = "このレポートでは、どのように格付けを予測していますか？500字で説明して下さい。"
result = qa({"query": query+"answer in Japanese"})
result

{'query': 'このレポートでは、どのように格付けを予測していますか？500字で説明して下さい。answer in Japanese',
 'result': ' このレポートでは、順序ロジット回帰とLGBMの2つのモデルを用いて、格付けを予測しています。まず、学習において、データセットの分割（訓練データとテストデータ）を行い、訓練データを使用してモデルを構築します。次に、テストデータを使用してモデルの予測性能を評価し、全サンプルの正答率を求めます。全サンプルの 正答率は、5クラス問題では、順序ロジット回帰が43.1％、LGBMが79.4',
 'source_documents': [Document(page_content='格付け分類モデルにおける機械学習の 応用： \n機械学習の 説明可能性 を高める 手法 \n橋本龍一郎 * \nryuuichirou.hashimoto@boj.or.jp  \n三浦翔 * \nkakeru.miura @boj.or.jp  \n吉崎康則 * \nyasunori.yoshizaki@boj.or.jp  \nNo.23-J-3 \n2023年3月 日本銀行  \n〒103-8660 日本郵便（株）日本橋郵便局私書箱 30号 \n*金融機構局\n日本銀行ワーキングペーパーシリーズは、日本銀行員および外部研究者の研究成果を\nとりまとめたもので、内外の研究機関、研究者等の有識者から幅広くコメントを頂戴す\nることを意図しています。ただし、論文の中で示された内容や意見は、日本銀行の公式\n見解を示すものではありません。  \nなお、ワーキングペーパーシリーズに対するご意見・ご質問や、掲載ファイルに関する\nお問い合わせは、執筆者までお寄せ下さい。  \n商用目的で転載・複製を行う場合は、予め日本銀行情報サービス局\n(post.prd8@boj.or.jp) までご相談下さい。転載・複製を行う場合は、出所を明記して\n下さい。  \n日本銀行ワーキングペーパーシリーズ', metadata={'source': 'wp23j03.pdf', 'page': 0}),
  Document(page_content='13 ３－１． 正答率による予測精度の評価  \nまず、正答率を確認する。 図表9は、予測された 格

In [11]:
query = "なぜ格付の予測が必要なのですか？"
result = qa({"query": query+"answer in Japanese"})
result

{'query': 'なぜ格付の予測が必要なのですか？answer in Japanese',
 'result': ' 機械学習を使用することにより、金融機関の業務をより効率的に管理することができるため、格付の予測が必要です。',
 'source_documents': [Document(page_content='格付け分類モデルにおける機械学習の 応用： \n機械学習の 説明可能性 を高める 手法 \n橋本龍一郎 * \nryuuichirou.hashimoto@boj.or.jp  \n三浦翔 * \nkakeru.miura @boj.or.jp  \n吉崎康則 * \nyasunori.yoshizaki@boj.or.jp  \nNo.23-J-3 \n2023年3月 日本銀行  \n〒103-8660 日本郵便（株）日本橋郵便局私書箱 30号 \n*金融機構局\n日本銀行ワーキングペーパーシリーズは、日本銀行員および外部研究者の研究成果を\nとりまとめたもので、内外の研究機関、研究者等の有識者から幅広くコメントを頂戴す\nることを意図しています。ただし、論文の中で示された内容や意見は、日本銀行の公式\n見解を示すものではありません。  \nなお、ワーキングペーパーシリーズに対するご意見・ご質問や、掲載ファイルに関する\nお問い合わせは、執筆者までお寄せ下さい。  \n商用目的で転載・複製を行う場合は、予め日本銀行情報サービス局\n(post.prd8@boj.or.jp) までご相談下さい。転載・複製を行う場合は、出所を明記して\n下さい。  \n日本銀行ワーキングペーパーシリーズ', metadata={'source': 'wp23j03.pdf', 'page': 0}),
  Document(page_content='1 格付け分類モデルにおける機械学習の 応用： \n機械学習の 説明可能性 を高める手法* \n \n橋本龍一郎†・三浦翔‡・吉崎康則§ \n \n2023年3月 \n \n【要旨】  \n \n近年、金融機関における様々な業務において機械学習の活用 が進んでいる。\n信用リスク管理の分野では、 クレジットスコアリングモデルや デフォルト率\nモデルに機械学習を 活用する事例がみられ始めて いる。本稿では、 格付け分\

In [12]:
query = "金融機関はどのように格付けを利用していますか？"
result = qa({"query": query+"answer in Japanese"})
result

{'query': '金融機関はどのように格付けを利用していますか？answer in Japanese',
 'result': ' 金融機関は、個別企業の財務データやマクロ経済変数から、個別債務者の信用力を予測するモデルを中心に、機械学習を活用して格付けを行っています。',
 'source_documents': [Document(page_content='格付け分類モデルにおける機械学習の 応用： \n機械学習の 説明可能性 を高める 手法 \n橋本龍一郎 * \nryuuichirou.hashimoto@boj.or.jp  \n三浦翔 * \nkakeru.miura @boj.or.jp  \n吉崎康則 * \nyasunori.yoshizaki@boj.or.jp  \nNo.23-J-3 \n2023年3月 日本銀行  \n〒103-8660 日本郵便（株）日本橋郵便局私書箱 30号 \n*金融機構局\n日本銀行ワーキングペーパーシリーズは、日本銀行員および外部研究者の研究成果を\nとりまとめたもので、内外の研究機関、研究者等の有識者から幅広くコメントを頂戴す\nることを意図しています。ただし、論文の中で示された内容や意見は、日本銀行の公式\n見解を示すものではありません。  \nなお、ワーキングペーパーシリーズに対するご意見・ご質問や、掲載ファイルに関する\nお問い合わせは、執筆者までお寄せ下さい。  \n商用目的で転載・複製を行う場合は、予め日本銀行情報サービス局\n(post.prd8@boj.or.jp) までご相談下さい。転載・複製を行う場合は、出所を明記して\n下さい。  \n日本銀行ワーキングペーパーシリーズ', metadata={'source': 'wp23j03.pdf', 'page': 0}),
  Document(page_content='2 １．はじめに  \n近年、金融機関における様々な業務において機械学習の活用が 進んでいる。Bank of \nEngland (2022) によれば、実務において機械学習を活用する英国の金融機関の割合は 7\n割強を占めるに至っており、顧客との関係構築、 マネー・ロンダリング防止対策や 各\n種リスク管理、 粉飾決算の検出 など幅広い業務分野で活用されている1。 \n信

In [13]:
query = "機械学習とロジット回帰ではどちらが精度が良いですか？ 400字以内で答えて下さい。"
result = qa({"query": query+"answer in Japanese"})
result

{'query': '機械学習とロジット回帰ではどちらが精度が良いですか？ 400字以内で答えて下さい。answer in Japanese',
 'result': ' アルゾンとカーボー（2021）や三浦ら（2019）の研究から、機械学習のLightGBMモデルは順序ロジット回帰よりも精度が高いことが分かりました。',
 'source_documents': [Document(page_content='13 ３－１． 正答率による予測精度の評価  \nまず、正答率を確認する。 図表9は、予測された 格付けを横軸に、実際の 格付けを\n縦軸に集計した 混合行列 （Confusion matrix ）である。正答率は、モデルが正しく 格付\nけを判別できた企業の割合であ り、ここでは、 混合行列の対角成分の和をサンプル数\nで割ることによって 、全サンプルの正答率 を定義する 。全サンプルの 正答率は、5ク\nラス問題では、 順序ロジット 回帰が43.1％、LGBMが79.4％となっており、 いずれの\nモデルでも実際の格付 けとの乖離はみられるが、 LGBMの正答率 の方が高い結果とな\nった。順序ロジット 回帰の混合行列を仔細にみると、両端（ A格以上、 CCC格以下）\nの正答率は 55～60％と相応に高い一方で、それ以外の 格付けでは正答率が 30％台と\n低いことが 分かる。この背景として、順序 ロジット 回帰では、2章で述べた財務指標\nと格付け間の非線形性や、財務指標 の分散と格付け間の相関といった 複雑な関係 性を\n捕捉することが難しいこと などが考えられる。 なお、2クラス問題 では、ロジット回\n帰が77.7％、LGBMが89.6％と、5クラス問題同様、 LGBMの方が正答率 が高い。  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n【順序ロジット 回帰】 \n①5クラス問題  \n【LGBM】 \n図表9 混合行列  \nA格以上 BBB格 BB格 B格CCC格以下 正答率（％）\nA格以上 5,992 569 139 48 15 88.6\nBBB格 681 6,965 723 223 75 80.4\nBB格 132 528 3,796 628 159 72.4\nB格 53 234 

In [14]:
query = "機械学習とロジット回帰ではどのぐらい精度に差がありますか？ 500字で答えて下さい。"
result = qa({"query": query+"answer in Japanese"})
result

{'query': '機械学習とロジット回帰ではどのぐらい精度に差がありますか？ 500字で答えて下さい。answer in Japanese',
 'result': ' 近年のデフォルト率モデル分野では、機械学習のLGBMの方が順序ロジット回帰より正答率が高く、5クラス問題では79.4％、2クラス問題では89.6％である一方、順序ロジット回帰では5クラス問題で43.1％、2クラス問題で77.7％となっていることから、機械学習とロジット回帰では精度にかなりの差があることがわかります。',
 'source_documents': [Document(page_content='13 ３－１． 正答率による予測精度の評価  \nまず、正答率を確認する。 図表9は、予測された 格付けを横軸に、実際の 格付けを\n縦軸に集計した 混合行列 （Confusion matrix ）である。正答率は、モデルが正しく 格付\nけを判別できた企業の割合であ り、ここでは、 混合行列の対角成分の和をサンプル数\nで割ることによって 、全サンプルの正答率 を定義する 。全サンプルの 正答率は、5ク\nラス問題では、 順序ロジット 回帰が43.1％、LGBMが79.4％となっており、 いずれの\nモデルでも実際の格付 けとの乖離はみられるが、 LGBMの正答率 の方が高い結果とな\nった。順序ロジット 回帰の混合行列を仔細にみると、両端（ A格以上、 CCC格以下）\nの正答率は 55～60％と相応に高い一方で、それ以外の 格付けでは正答率が 30％台と\n低いことが 分かる。この背景として、順序 ロジット 回帰では、2章で述べた財務指標\nと格付け間の非線形性や、財務指標 の分散と格付け間の相関といった 複雑な関係 性を\n捕捉することが難しいこと などが考えられる。 なお、2クラス問題 では、ロジット回\n帰が77.7％、LGBMが89.6％と、5クラス問題同様、 LGBMの方が正答率 が高い。  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n【順序ロジット 回帰】 \n①5クラス問題  \n【LGBM】 \n図表9 混合行列  \nA格以上 BBB格 BB格 B格CCC格以下 正答率（％）\nA格以上 5,992 569 139 4